# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 29 2022 8:33AM,243042,10,0085914664,ISDIN Corporation,Released
1,Jul 29 2022 8:33AM,243042,10,0085914719,ISDIN Corporation,Released
2,Jul 29 2022 8:33AM,243042,10,0085914752,ISDIN Corporation,Released
3,Jul 29 2022 8:33AM,243042,10,0085914756,ISDIN Corporation,Released
4,Jul 29 2022 8:33AM,243042,10,0085914746,ISDIN Corporation,Released
5,Jul 29 2022 8:33AM,243042,10,0085914771,ISDIN Corporation,Released
6,Jul 29 2022 8:33AM,243042,10,0085914772,ISDIN Corporation,Released
7,Jul 29 2022 8:33AM,243042,10,0085914773,ISDIN Corporation,Released
8,Jul 29 2022 8:33AM,243042,10,0085914784,ISDIN Corporation,Released
9,Jul 29 2022 8:33AM,243042,10,0085914787,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,243038,Released,1
21,243039,Released,57
22,243040,Released,7
23,243041,Released,1
24,243042,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243038,NaN,NaN,1.0
243039,NaN,NaN,57.0
243040,NaN,NaN,7.0
243041,NaN,NaN,1.0
243042,NaN,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242964,17.0,10.0,7.0
242965,0.0,0.0,1.0
242967,0.0,0.0,1.0
242976,0.0,0.0,1.0
243003,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242964,17,10,7
242965,0,0,1
242967,0,0,1
242976,0,0,1
243003,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242964,17,10,7
1,242965,0,0,1
2,242967,0,0,1
3,242976,0,0,1
4,243003,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242964,17,10,7
1,242965,,,1
2,242967,,,1
3,242976,,,1
4,243003,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation
15,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation
22,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc."
23,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation
80,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation
111,Jul 29 2022 8:23AM,243038,22,"NBTY Global, Inc."
112,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated"
113,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.
118,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.
119,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,,,15
1,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,,,7
2,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",,,1
3,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation,,,57
4,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation,,,31
5,Jul 29 2022 8:23AM,243038,22,"NBTY Global, Inc.",,,1
6,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",,,1
7,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,,,5
8,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,,,1
9,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,15,,
1,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,7,,
2,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",1,,
3,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation,57,,
4,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation,31,,
5,Jul 29 2022 8:23AM,243038,22,"NBTY Global, Inc.",1,,
6,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",1,,
7,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,5,,
8,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,1,,
9,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,15,,
1,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,7,,
2,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",1,,
3,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation,57,,
4,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation,31,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,15.0,NaN,NaN
1,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,7.0,NaN,NaN
2,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation,57.0,NaN,NaN
4,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation,31.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:33AM,243042,10,ISDIN Corporation,15.0,0.0,0.0
1,Jul 29 2022 8:31AM,243040,10,ISDIN Corporation,7.0,0.0,0.0
2,Jul 29 2022 8:30AM,243041,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 29 2022 8:30AM,243039,10,ISDIN Corporation,57.0,0.0,0.0
4,Jul 29 2022 8:27AM,243030,10,ISDIN Corporation,31.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1701257,114.0,0.0,0.0
12,243024,0.0,1.0,0.0
16,728935,3.0,0.0,0.0
20,972052,32.0,10.0,17.0
21,729019,3.0,0.0,0.0
22,1215087,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1701257,114.0,0.0,0.0
1,12,243024,0.0,1.0,0.0
2,16,728935,3.0,0.0,0.0
3,20,972052,32.0,10.0,17.0
4,21,729019,3.0,0.0,0.0
5,22,1215087,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,114.0,0.0,0.0
1,12,0.0,1.0,0.0
2,16,3.0,0.0,0.0
3,20,32.0,10.0,17.0
4,21,3.0,0.0,0.0
5,22,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,114.0
1,12,Released,0.0
2,16,Released,3.0
3,20,Released,32.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20,21,22
Status,,,,,,
Completed,0.0,0.0,0.0,17.0,0.0,0.0
Executing,0.0,1.0,0.0,10.0,0.0,0.0
Released,114.0,0.0,3.0,32.0,3.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,0.0,1.0,0.0,10.0,0.0,0.0
2,Released,114.0,0.0,3.0,32.0,3.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,0.0,1.0,0.0,10.0,0.0,0.0
2,Released,114.0,0.0,3.0,32.0,3.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()